In [13]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [14]:
#Read in CSV
weather_df = pd.read_csv("Output_CSV/weather_df.csv")
weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed
0,0,Sabang,ID,5.8933,95.3214,100,71,83.01,8.81
1,1,Nikolskoye,RU,59.7035,30.7861,94,93,44.49,8.14
2,2,Tigil,RU,57.8000,158.6667,8,82,40.08,3.38
3,3,Pasighat,IN,28.0667,95.3333,26,55,93.09,4.07
4,4,Kavieng,PG,-2.5744,150.7967,99,78,80.80,25.99
...,...,...,...,...,...,...,...,...,...
588,588,Nagato,JP,34.3833,131.2000,41,82,75.15,15.82
589,589,Kongolo,CD,-5.3833,27.0000,25,26,100.13,3.78
590,590,Mangan,IN,27.5167,88.5333,100,83,81.23,4.32
591,591,Yenagoa,NG,4.9247,6.2642,100,82,79.56,1.43


In [29]:
#Set coordinates for markers and define humidity for heatmap
coordinates = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']

# Create a heatmap marker layer and display
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(coordinates, weights=humidity, max_intensity=100, point_radius=5, dissipating=False )
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
#Revise weather dataframe to cities with only ideal conditions
ideal_weather = weather_df.loc[(weather_df['Temperature'] <83) & (weather_df['Temperature'] >72)
                               & (weather_df['Wind Speed'] < 3) & (weather_df['Cloudiness'] <5)]
    
#Display revised dataframe
ideal_weather

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed
151,151,Mayo,US,38.8876,-76.5119,1,88,72.43,1.99
276,276,Wilmington,US,34.2257,-77.9447,1,95,72.10,0.00
301,301,Havelock,US,34.8791,-76.9013,0,95,72.34,2.71
312,312,Topolobampo,MX,25.6000,-109.0500,4,75,78.06,2.82
387,387,Saint George,US,37.1041,-113.5841,1,19,74.68,0.00
461,461,Lixourion,GR,38.2019,20.4314,0,73,78.80,0.00
536,536,Bijela,BA,43.6086,17.9519,3,24,74.10,2.77
539,539,Nador,MA,35.1740,-2.9287,0,68,73.65,1.14


In [31]:
#Make copy of ideal_weather df to avoid splice warning
paradise_df = ideal_weather.copy()

#Create new column in dataframe to hold hotel info
paradise_df['Hotel Name'] = ""
paradise_df["Coordinates"] = tuple(zip(paradise_df['Latitude'], paradise_df['Longitude']))
paradise_df.head(10)

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Hotel Name,Coordinates
151,151,Mayo,US,38.8876,-76.5119,1,88,72.43,1.99,,"(38.8876, -76.5119)"
276,276,Wilmington,US,34.2257,-77.9447,1,95,72.10,0.00,,"(34.2257, -77.9447)"
301,301,Havelock,US,34.8791,-76.9013,0,95,72.34,2.71,,"(34.8791, -76.9013)"
312,312,Topolobampo,MX,25.6000,-109.0500,4,75,78.06,2.82,,"(25.6, -109.05)"
387,387,Saint George,US,37.1041,-113.5841,1,19,74.68,0.00,,"(37.1041, -113.5841)"
461,461,Lixourion,GR,38.2019,20.4314,0,73,78.80,0.00,,"(38.2019, 20.4314)"
536,536,Bijela,BA,43.6086,17.9519,3,24,74.10,2.77,,"(43.6086, 17.9519)"
539,539,Nador,MA,35.1740,-2.9287,0,68,73.65,1.14,,"(35.174, -2.9287)"


In [32]:
# find the closest hotel to each of the city coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in paradise_df.iterrows():
    params = {
        "location": f'{row[3]},{row[4]}',
        "radius": 5000,
        "type": 'lodging',
        "key": gkey
    }
    response = requests.get(base_url, params=params).json()
    
#Set exception for no results error, otherwise populate Hotel Name column with Hotel Name
    try:
        paradise_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        paradise_df.loc[index, 'Hotel Name'] = "NaN"
        
#Display dataframe with hotels column populated
paradise_df.head(10)

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Hotel Name,Coordinates
151,151,Mayo,US,38.8876,-76.5119,1,88,72.43,1.99,Resorts Resource Group International,"(38.8876, -76.5119)"
276,276,Wilmington,US,34.2257,-77.9447,1,95,72.10,0.00,"Hotel Ballast Wilmington, Tapestry Collection ...","(34.2257, -77.9447)"
301,301,Havelock,US,34.8791,-76.9013,0,95,72.34,2.71,Sherwood Motel,"(34.8791, -76.9013)"
312,312,Topolobampo,MX,25.6000,-109.0500,4,75,78.06,2.82,Hotel Marina Topolobampo,"(25.6, -109.05)"
387,387,Saint George,US,37.1041,-113.5841,1,19,74.68,0.00,Best Western Plus Abbey Inn,"(37.1041, -113.5841)"
461,461,Lixourion,GR,38.2019,20.4314,0,73,78.80,0.00,Hotel Summery,"(38.2019, 20.4314)"
536,536,Bijela,BA,43.6086,17.9519,3,24,74.10,2.77,Garden City Konjic,"(43.6086, 17.9519)"
539,539,Nador,MA,35.1740,-2.9287,0,68,73.65,1.14,Hôtel La Giralda,"(35.174, -2.9287)"


In [33]:
#Remove any rows where a hotel was not found
paradise_df.dropna()

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Hotel Name,Coordinates
151,151,Mayo,US,38.8876,-76.5119,1,88,72.43,1.99,Resorts Resource Group International,"(38.8876, -76.5119)"
276,276,Wilmington,US,34.2257,-77.9447,1,95,72.10,0.00,"Hotel Ballast Wilmington, Tapestry Collection ...","(34.2257, -77.9447)"
301,301,Havelock,US,34.8791,-76.9013,0,95,72.34,2.71,Sherwood Motel,"(34.8791, -76.9013)"
312,312,Topolobampo,MX,25.6000,-109.0500,4,75,78.06,2.82,Hotel Marina Topolobampo,"(25.6, -109.05)"
387,387,Saint George,US,37.1041,-113.5841,1,19,74.68,0.00,Best Western Plus Abbey Inn,"(37.1041, -113.5841)"
461,461,Lixourion,GR,38.2019,20.4314,0,73,78.80,0.00,Hotel Summery,"(38.2019, 20.4314)"
536,536,Bijela,BA,43.6086,17.9519,3,24,74.10,2.77,Garden City Konjic,"(43.6086, 17.9519)"
539,539,Nador,MA,35.1740,-2.9287,0,68,73.65,1.14,Hôtel La Giralda,"(35.174, -2.9287)"


In [34]:
#Create info box for marker labels
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in paradise_df.iterrows()]

#Set marker locations
marker_locations = paradise_df[['Latitude', 'Longitude']]

#Configure and display map
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))